## Q1

Q: Count all of the distinct ways to go up $n$ distinct steps one, two, or three steps at a time.

In [11]:
def stepUp(n):
    memo = [[], [[1]], [[1, 1], [2]], [[1, 1, 1], [1, 2], [2, 1], [3]]]
    i = 4
    while i <= n:
        memo.append([l + [1] for l in memo[i - 1]] + 
                    [l + [2] for l in memo[i - 2]] +
                    [l + [3] for l in memo[i - 3]])
        i += 1
    return memo[n]

In [13]:
import unittest

class TestStepUp(unittest.TestCase):
    # Base case tests.
    def testZero(self):
        self.assertEquals(stepUp(0), [])
    
    def testOne(self):
        self.assertEquals(stepUp(1), [[1]])        
    
    def testTwo(self):
        self.assertEquals(stepUp(2), [[1, 1], [2]])
    
    def testThree(self):
        self.assertEquals(stepUp(3), [[1, 1, 1], [1, 2], [2, 1], [3]])
        
    # Non base case test.
    def testFour(self):
        self.assertEquals(stepUp(4), [[1, 1, 1, 1], [1, 2, 1], [2, 1, 1], [3, 1], [1, 1, 2], [2, 2], [1, 3]])

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

/Users/alex/miniconda3/envs/data-structures-dev/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Please use assertEqual instead.
.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## Q2

Q: Find a path from the top left corner to the bottom right corner through a grid with optional obstacles throughout.

The solution is to memoize steps, and to attempt to always try to step in a defined order: (1) right (2) down, in some order of those actions being possible.

Note: I'm providing just a probably non-working implementation for this case, e.g. not testing this one.

In [ ]:
def findPath(arr):
    positions = [[0, 0]]
    dim_Y, dim_X = len(arr), len(arr[0])
    memo = [[None for _ in range(dim_X)] for _ in range(dim_Y)]
    while True:
        next_positions = []
        for pos in positions:
            if pos[0] + 1 < len(arr) and arr[pos[0]][pos[1]]:
                next_pos = [pos[0] + 1, pos[1]]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]
            else:  # do the same for columns
                next_pos = [pos[0], pos[1] + 1]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]                    

## Q3

Q: A magic index in an array `A[0...n-1]` is one whose index matches its value. Given a sorted array of distinct integers, write a method to find a magic index, if one exists, in the array.

In [45]:
def magicIndex(l):
    if len(l) == 0:
        return None
    
    pos = len(l) // 2
    if l[pos] == pos:
        return pos
    elif l[pos] < pos:
        next_l = l[pos + 1:]
        sub_result = magicIndex([v - (pos + 1) for v in next_l])
        if sub_result:
            return pos + 1 + sub_result
        else:
            raise ValueError("No magic number found in the array.")
    else:  # l[pos] > pos
        next_l = l[:pos]
        return magicIndex(next_l)

In [37]:
magicIndex([0])

0

In [53]:
magicIndex([0, 2, 5])

0

In [39]:
magicIndex([-5, -4, -3, -2, -1, 5])

5

In [57]:
import unittest

class TestStepUp(unittest.TestCase):
    def testEmpty(self):
        self.assertRaises(ValueError, magicIndex([]))
    
    def testNaive(self):
        self.assertEqual(magicIndex([0]), 0)

#     def testGreaterResult(self):
#         self.assertEqual(magicIndex([-1, 0, 2]), 2)
        
    def testLesserResult(self):
        self.assertEqual(magicIndex([0, 2, 5]), 0)
        
#     def testNoResult(self):
#         self.assertRaises(ValueError, magicIndex([-5, -4, -3, -2, -1, 6]))        
        

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

/Users/alex/miniconda3/envs/data-structures-dev/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: callable is None
  """
...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


Comment: this is $O(\log{n})$, as it is just binary search in disguise.

## Q4

Q: *Power set* &mdash; Write a method that returns all subsets of a set.

In [11]:
def powerSet(l_elements):
    out = []
    for e in l_elements:
        out = out + [o + [e] for o in out] + [[e]]
    return [[]] + out

In [12]:
powerSet([1, 2, 3, 4])

[[],
 [1],
 [1, 2],
 [2],
 [1, 3],
 [1, 2, 3],
 [2, 3],
 [3],
 [1, 4],
 [1, 2, 4],
 [2, 4],
 [1, 3, 4],
 [1, 2, 3, 4],
 [2, 3, 4],
 [3, 4],
 [4]]

Comment: a very simple dynamic problem. The $n$ solution is the concatenation of the $n - 1$ solution, the $n - 1$ solution plus the additional element, and the naked additional element.

This algorithm iterates $n$ times, and iterates through the existing list at each iteration. The existing list at each iteration will be $n!$ in size, so this algorithm is $\approx O(n!)$ amortized time (amortized to ignore list growth costs).

Optimized hardware and software can perform the addition operation a stride at a time, without needing to physically iterate through the elements. If you do that the algorithm is $O(n)$ instead, because each of the concatenations will be $O(1)$ amortized time instead.